In [1]:
include("distances/new_distance.jl")
include("distances/distance_Wasserstein.jl")
using RCall # to call R functions


In [2]:
R"""

library(transport)

# Measures are given as a nTop x nBottom x 2 array
# nTop = n in the article 
# nBottom = m in the article 
# nGrid = M in the article 
# a[,,1] -> location of the atom 
# a[,,2] -> mass of the atom 

# First simplest 1D Wasserstein distance between measures with the same number of atom and equal measures

wasserstein1DUniform <- function(atoms1, atoms2, p) {
  # atoms1 and atoms2 only list of position 
  # p is the exponent 
  if (length(atoms1) == length(atoms2)) {

    diff = abs(sort(atoms1) - sort(atoms2))
    return(mean((diff^p))^(1/p))
  } else {
    print("ERROR: not the same number of atoms")
    return(-1.0)
  }
}


ww <- function(measure1, measure2, p) {
  # Assuming that the number of atoms at the lower level is the same in each measure 
  
  s1 <- dim(measure1)
  s2 <- dim(measure2)
  
  if (s1[2] != s2[2]) {
    cat("PROBLEM OF DIMENSION: each lower measure should have the same dimension\n")
    return(-1.0)
  } else {
    

    # Extract dimensions
    m1 <- s1[1]
    m2 <- s2[1]
    n  <- s1[2]
    
    # Compute matrix of pairwise distances which will be a cost function 
    
    C <- matrix(0, m1, m2)
    for (i in 1:m1) {
      for (j in 1:m2) {
        C[i, j] <- wasserstein1DUniform(measure1[i, ], measure2[j, ], p)^p
      }
    }
    

    # Build the weights: uniform 
    weight1 <- rep(1 / m1, m1)
    weight2 <- rep(1 / m2, m2)
    
    # Solving the optimal transport problem 
    output <-(wasserstein(weight1, weight2, p = 1, costm = C, prob = TRUE))^(1 / p)
    return(output)
  }
}


"""


RObject{ClosSxp}
function (measure1, measure2, p) 
{
    s1 <- dim(measure1)
    s2 <- dim(measure2)
    if (s1[2] != s2[2]) {
        cat("PROBLEM OF DIMENSION: each lower measure should have the same dimension\n")
        return(-1)
    }
    else {
        m1 <- s1[1]
        m2 <- s2[1]
        n <- s1[2]
        C <- matrix(0, m1, m2)
        for (i in 1:m1) {
            for (j in 1:m2) {
                C[i, j] <- wasserstein1DUniform(measure1[i, ], 
                  measure2[j, ], p)^p
            }
        }
        weight1 <- rep(1/m1, m1)
        weight2 <- rep(1/m2, m2)
        output <- (wasserstein(weight1, weight2, p = 1, costm = C, 
            prob = TRUE))^(1/p)
        return(output)
    }
}


In [3]:
function ww_from_r(measure1::Matrix{Float64}, measure2::Matrix{Float64}, p::Int);
    # This function computes Wasserstein distance between two samples of Normal distributions using R.

                         


    @rput measure1 measure2 p

    R"""
    # if (!requireNamespace("frechet", quietly = TRUE)) {
    #   install.packages("frechet", repos="https://cloud.r-project.org")
    # }
    dist_ww = ww(measure1, measure2, p)
    """
    @rget dist_ww

    return dist_ww
end








ww_from_r (generic function with 1 method)

In [4]:
n = 10


m = 4
measure_1_r = 10*randn(n,m)
measure_2_r = 10*randn(n,m)
a = minimum([minimum(measure_1_r), minimum(measure_2_r)])
b = maximum([maximum(measure_1_r), maximum(measure_2_r)])
measure_1_julia = zeros(n,m,2)
measure_2_julia = zeros(n,m,2)


measure_1_julia[:,:,1] = measure_1_r
measure_1_julia[:,:,2] = fill(1/m, (n,m))

measure_2_julia[:,:,1] = measure_2_r
measure_2_julia[:,:,2] = fill(1/m, (n,m))

p = 1

dist_ww_julia = ww(measure_1_r, measure_2_r, p)

dist_ww_r = ww_from_r(measure_1_r, measure_2_r, p)
println("Wasserstein distance from R: ", dist_ww_r)
println("Wasserstein distance from Julia: ", dist_ww_julia)


Wasserstein distance from R: 4.271955380846802
Wasserstein distance from Julia: 4.2719553514424184
